In [0]:
from google.colab import files, drive
import pandas as pd
import numpy as np
import sys
import os
import re

In [0]:
def progressBar(cur, end, length=25):
  percentage = cur/end
  loading = '.' * (int(round(percentage * length))-1)
  spaces = ' ' * (length - len(loading))

  sys.stdout.write("\rLoading: [{0}] {1}%".format(loading + spaces, int(round(percentage * 100))))
  sys.stdout.flush()

In [0]:
os.environ['KAGGLE_USERNAME'] = 'vadmanuel'
os.environ['KAGGLE_KEY'] = 'key'
!kaggle competitions download -c ucfai-dsg-quals-fa19 --force

  0% 0.00/829k [00:00<?, ?B/s]
100% 829k/829k [00:00<00:00, 56.4MB/s]
  0% 0.00/281k [00:00<?, ?B/s]
100% 281k/281k [00:00<00:00, 86.7MB/s]


In [0]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
df = pd.read_csv('/content/train.csv')

In [0]:
columns_set = set(df.columns) - set(['ID', 'DOB', 'SALARY', 'ENDU_LEVEL', 'DIST_FRM_COAST', 'MNTLY_TRAVEL', 'Dist_Coast'])
sets = {}
for c in columns_set:
  temp_list = list(set(df.loc[:df.shape[0],c]))
  sets[c] = temp_list

In [0]:
df_copy = pd.DataFrame(columns=df.columns)

re_endu_level_item = re.compile('(\d+)(?: to )(\d+)|(\d+)(?:-)(\d+)|(\d+)(?:\+)')
re_dist_frm_coast_item = re.compile('(\d+)(?:-)(\d+)')
re_mntly_trave_time = re.compile('(\d+)(?:-)(\d+)|(\d+)(?:\+)')
re_salary = re.compile('(\d+)(?:[kK]?)(?:-)(\d+)(?:[kK]?)|(\d+)(?:M-)(\d+)(?:Million)')
re_dob = re.compile('(\d+)(?:/)(\d+)(?:/)(\d+)')

In [0]:
# dropped last row since it was just empty
for i in range(df.shape[0]-1):
  progressBar(i, df.shape[0])
  for c in columns_set:
    df_copy.loc[i, c] = sets[c].index(df.loc[i, c])

  dob_item = df.loc[i, 'DOB']
  if type(dob_item) is not str:
    df_copy.drop([i])
    continue
  check = re.search(re_dob, dob_item)
  if check.group():
    df_copy.loc[i, 'DOB'] = np.array([int(check.group(1)), int(check.group(2)), int(check.group(3))], dtype=np.int64)
  else:
    df_copy.drop([i])

  salary_item = df.loc[i, 'SALARY']
  if type(salary_item) is not str:
    df_copy.drop([i])
    continue
  check = re.search(re_salary, salary_item)
  if check.group(1) and check.group(2):
    df_copy.loc[i, 'SALARY'] = np.array([int(check.group(1)), int(check.group(2))], dtype=np.int64)
  elif check.group(3) and check.group(4):
    df_copy.loc[i, 'SALARY'] = np.array([int(check.group(3) + '000'), int(check.group(4) + '000')], dtype=np.int64)
  else:
    df_copy.drop([i])

  endu_level_item = df.loc[i,'ENDU_LEVEL']
  if type(endu_level_item) is not str:
    df_copy.drop([i])
    continue
  check = re.search(re_endu_level_item, endu_level_item)
  if check.group(1) and check.group(2):
    df_copy.loc[i, 'ENDU_LEVEL'] = df_copy.loc[i, 'MNTLY_TRAVEL'] = np.array([int(check.group(1)), int(check.group(2))], dtype=np.int64)
  elif check.group(3) and check.group(4):
    df_copy.loc[i, 'ENDU_LEVEL'] = df_copy.loc[i, 'MNTLY_TRAVEL'] = np.array([int(check.group(3)), int(check.group(4))], dtype=np.int64)
  elif check.group(5):
    df_copy.loc[i, 'ENDU_LEVEL'] = df_copy.loc[i, 'MNTLY_TRAVEL'] = np.array([int(check.group(5))], dtype=np.int64)
  else:
    df_copy.drop([i])

  
  dist_frm_coast_item = df.loc[i,'DIST_FRM_COAST']
  if type(dist_frm_coast_item) is not str:
    df_copy.drop([i])
    continue
  check = re.search(re_dist_frm_coast_item, dist_frm_coast_item)
  if check.group():
    df_copy.loc[i, 'DIST_FRM_COAST'] = df_copy.loc[i, 'MNTLY_TRAVEL'] = np.array([int(check.group(1)), int(check.group(2))], dtype=np.int64)
  else:
    df_copy.drop([i])

  mntly_travel_item = df.loc[i,'MNTLY_TRAVEL']
  if type(mntly_travel_item) is not str:
    df_copy.drop([i])
    continue
  check = re.search(re_mntly_trave_time, mntly_travel_item)
  if check.group(1) and check.group(2):
    df_copy.loc[i, 'MNTLY_TRAVEL'] = np.array([int(check.group(1)), int(check.group(2))], dtype=np.int64)
  elif check.group(3):
    df_copy.loc[i, 'MNTLY_TRAVEL'] = np.array([int(check.group(3))], dtype=np.int64)
  else:
    df_copy.drop([i])
  
  dist_coast_item = df.loc[i,'Dist_Coast']
  if type(dist_coast_item) not in [np.int64, np.float64]:
    df_copy.drop([i])
    continue
  df_copy.loc[i, 'Dist_Coast'] = np.int64(dist_coast_item)

  id_item = df.loc[i, 'ID']
  if type(id_item) not in [np.int64, np.float64]:
    df_copy.drop([i])
    continue
  df_copy.loc[i, 'ID'] = np.int64(id_item)

df_copy.tail(10)

Loading: [........................ ] 100%

,ID,DOB,M_STATUS,SALARY,EDU_DATA,EMP_DATA,REL_ORIEN,FAV_TV,PREF_CAR,GENDER,FAV_CUIS,FAV_MUSIC,ENDU_LEVEL,FAV_SPORT,FAV_COLR,NEWS_SOURCE,DIST_FRM_COAST,MNTLY_TRAVEL,GEN_MOVIES,FAV_SUBJ,ALCOHOL,FAV_SUPERHERO,Dist_Coast,Class
3509,8355,"[5, 26, 1989]",3,"[0, 100]",4,1,3,4,13,2,5,2,"[15, 18]",7,1,7,"[10, 25]",[2000],0,3,2,7,304,1
3510,8356,"[3, 24, 1981]",1,"[500, 700]",3,2,1,0,9,1,4,4,"[6, 10]",4,6,3,"[10, 25]","[250, 500]",3,10,4,1,1336,0
3511,8357,"[9, 21, 1972]",2,"[500, 700]",2,3,2,4,9,1,3,2,"[15, 18]",6,8,8,"[25, 40]","[500, 1000]",6,4,7,2,736,1
3512,8358,"[11, 20, 1964]",3,"[300, 500]",3,1,2,5,1,2,6,3,"[3, 5]",7,9,6,"[10, 25]","[250, 500]",3,10,6,1,1020,0
3513,8359,"[8, 5, 1962]",3,"[500, 700]",1,2,2,4,4,2,6,5,"[10, 12]",7,8,8,"[60, 80]","[250, 500]",0,3,2,7,1249,1
3514,8360,"[8, 14, 1982]",2,"[300, 500]",4,2,2,0,1,2,8,3,"[6, 10]",1,9,4,"[60, 80]",[2000],2,1,4,2,130,0
3515,8361,"[4, 28, 1972]",3,"[100, 300]",4,2,3,8,3,2,8,1,"[12, 15]",8,1,7,"[0, 10]",[2000],0,3,4,7,417,1
3516,8362,"[11, 11, 1972]",3,"[100, 300]",3,2,2,7,14,2,8,8,"[6, 10]",7,4,3,"[0, 10]","[0, 250]",3,10,7,1,391,0
3517,8363,"[9, 4, 1972]",3,"[0, 100]",2,3,1,7,10,1,4,4,"[6, 10]",2,4,4,"[40, 60]","[1500, 2000]",7,7,5,6,497,1
3518,8364,"[5, 26, 1979]",0,"[0, 100]",2,3,2,0,10,2,9,3,"[3, 5]",6,4,3,"[0, 10]","[0, 250]",3,10,3,1,469,1


In [0]:
df_copy.to_csv('gdrive/My Drive/prep_train.csv', index=False)
files.download('prep_train.csv')

In [0]:
df_copy

,ID,DOB,M_STATUS,SALARY,EDU_DATA,EMP_DATA,REL_ORIEN,FAV_TV,PREF_CAR,GENDER,FAV_CUIS,FAV_MUSIC,ENDU_LEVEL,FAV_SPORT,FAV_COLR,NEWS_SOURCE,DIST_FRM_COAST,MNTLY_TRAVEL,GEN_MOVIES,FAV_SUBJ,ALCOHOL,FAV_SUPERHERO,Dist_Coast,Class
0,4846,"[3, 2, 1991]",3,"[100, 300]",3,2,2,9,14,2,3,4,"[6, 10]",7,6,3,"[0, 10]","[0, 250]",0,7,2,7,202,1
1,4847,"[11, 7, 1977]",1,"[100, 300]",2,3,1,5,3,1,2,4,"[6, 10]",4,6,3,"[10, 25]","[250, 500]",5,10,6,3,1454,1
2,4848,"[7, 4, 1964]",3,"[300, 500]",3,1,2,0,1,1,4,4,"[6, 10]",7,6,3,"[10, 25]","[0, 250]",5,10,4,3,623,1
3,4849,"[2, 11, 1983]",3,"[0, 100]",2,2,1,8,10,2,9,1,"[12, 15]",2,8,1,"[25, 40]","[0, 250]",3,3,4,1,164,1
4,4850,"[2, 12, 1981]",2,"[300, 500]",1,3,2,4,4,1,4,2,"[15, 18]",1,3,2,"[40, 60]","[0, 250]",4,6,2,4,1423,0
5,4851,"[2, 16, 1977]",3,"[500, 700]",4,1,1,7,9,1,3,4,"[6, 10]",2,9,4,"[10, 25]","[0, 250]",7,7,5,6,1480,0
6,4852,"[5, 22, 1981]",2,"[500, 700]",3,1,2,8,1,2,5,2,"[15, 18]",1,8,2,"[40, 60]","[250, 500]",4,6,4,4,143,1
7,4853,"[11, 22, 1970]",3,"[300, 500]",3,2,2,0,12,1,4,4,"[6, 10]",6,4,4,"[10, 25]","[0, 250]",5,10,3,3,45,0
8,4854,"[10, 27, 1981]",3,"[100, 300]",2,2,2,8,14,2,5,2,"[15, 18]",6,3,1,"[25, 40]","[500, 1000]",0,3,1,7,944,1
9,4855,"[4, 13, 1994]",2,"[700, 1]",4,3,2,0,12,2,8,3,"[6, 10]",1,9,6,"[0, 10]","[0, 250]",0,7,3,7,1102,0
